In [6]:
import pandas as pd
import torch
import chemistry_vae_selfies

In [3]:
data = pd.read_csv('./datasets/PropsQM9/listprops.csv', index_col=None)

In [7]:
# here we want to read in data and tranform them into one_hot
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "./datasets/"
file_name = "SelectedSMILES_QM9.txt"

full_path = folder_path + file_name

selfies_list, selfies_alphabet, largest_selfies_len, smiles_list, smiles_alphabet, largest_smiles_len = chemistry_vae_selfies.get_selfie_and_smiles_encodings_for_dataset(full_path)

--> Translating SMILES to SELFIES...


Finished translating SMILES to SELFIES.
selfies aplhabet: ['[=Branch2]', '[#Branch1]', '[Ring2]', '[=C]', '[nop]', '[Branch2]', '[#C]', '[#Branch2]', '[Ring1]', '[N]', '[=N]', '[=Branch1]', '[C]', '[F]', '[Branch1]', '[#N]', '[O]', '[=O]']
smiles aplhabet: ['N', '#', 'C', '(', 'F', '=', 'O', '3', '5', '2', '1', '4', ')', ' ']


In [8]:
selfies_alphabet = ['[#Branch2]', '[Ring2]', '[Branch2]', '[=Branch2]', '[O]', '[=O]', '[=C]', '[=N]', '[#Branch1]', '[=Branch1]', '[nop]', '[N]', '[Branch1]', '[F]', '[#C]', '[#N]', '[Ring1]', '[C]']


In [9]:
# define source file location
file_to_load =  "./saved_models_RNN/"
# training file name encoder
training_file_nameE = "300/E"
# training file name decoder
training_file_nameD = "300/D"
# load data
#load_data_trained = file_to_load + training_file_nameE
# Alphabet has 18 letters, largest molecule is 21 letters. (build this as an output function later ... )
largest_selfies_len_dataset = largest_selfies_len
largest_smiles_len_dataset = largest_smiles_len

#in_dimension = len(selfies_alphabet)*largest_selfies_len
in_dimension = len(smiles_alphabet)*largest_smiles_len



# load the trained encoder
vae_encoder = torch.load(file_to_load + training_file_nameE) #, map_location=torch.device(device="cpu"))
#print(vae_encoder)

# load the trained decoder
vae_decoder = torch.load(file_to_load + training_file_nameD) #, map_location=torch.device(device="cpu"))
#print(vae_decoder)

In [83]:
def create_random_latent_space_vector(largest_selfies_len,selfies_alphabet_in):

    # Random input tensor for tests
    in_dimension_input = largest_selfies_len*len(selfies_alphabet_in)
    x = torch.randn(in_dimension_input).unsqueeze(0)

    z =set()
    vae_encoder.eval()
    vae_decoder.eval()
    z, mu, log_var = vae_encoder(x)

    return z.unsqueeze(0), mu, log_var.unsqueeze(0)

In [84]:
z, mu, log_var = create_random_latent_space_vector(largest_selfies_len, selfies_alphabet)

In [129]:
list_var = []
list_var.append(log_var)

tensor([[[-7.1405, -5.7359, -8.3026, -8.4360, -7.7541, -7.6566, -7.2899,
          -7.0162, -8.5449, -7.1038, -8.4080, -9.1792, -8.8428, -8.0314,
          -8.9814, -8.9128, -7.1589, -9.4305, -8.6080, -8.2441, -8.9749,
          -8.1896, -8.2922, -7.6497, -9.4049]]], grad_fn=<UnsqueezeBackward0>)


In [131]:
log_var

log_vars_valid = [item[0].detach().squeeze(0) for item in log_var]


In [132]:
log_vars_valid

[tensor([-7.1405, -5.7359, -8.3026, -8.4360, -7.7541, -7.6566, -7.2899, -7.0162,
         -8.5449, -7.1038, -8.4080, -9.1792, -8.8428, -8.0314, -8.9814, -8.9128,
         -7.1589, -9.4305, -8.6080, -8.2441, -8.9749, -8.1896, -8.2922, -7.6497,
         -9.4049])]

In [120]:
import math

In [133]:
log_var2 = (math.e**(log_vars_valid)[0])

print(log_var2)


tensor([7.9237e-04, 3.2280e-03, 2.4786e-04, 2.1691e-04, 4.2899e-04, 4.7290e-04,
        6.8243e-04, 8.9722e-04, 1.9453e-04, 8.2197e-04, 2.2307e-04, 1.0316e-04,
        1.4441e-04, 3.2510e-04, 1.2572e-04, 1.3466e-04, 7.7793e-04, 8.0242e-05,
        1.8264e-04, 2.6281e-04, 1.2655e-04, 2.7753e-04, 2.5047e-04, 4.7621e-04,
        8.2319e-05])


In [134]:
log_vars_valid = (log_var2 - log_var2[(log_var2).argmin()]) / (log_var2[(log_var2).argmax()] - log_var2[(log_var2).argmin()])



In [126]:
train_size = 24

In [135]:
log_var_rep_train = log_vars_valid[:train_size]


In [128]:
log_var_rep_train

tensor([0.2262, 1.0000, 0.0533, 0.0434, 0.1108, 0.1247, 0.1913, 0.2595, 0.0363,
        0.2356, 0.0454, 0.0073, 0.0204, 0.0778, 0.0144, 0.0173, 0.2216, 0.0000,
        0.0325, 0.0580, 0.0147, 0.0627, 0.0541, 0.1258])

In [117]:
print(log_vars_valid.view(-1, 25))

tensor([[0.4010, 1.0000, 0.1418, 0.1206, 0.2456, 0.2672, 0.3587, 0.4387, 0.1043,
         0.4119, 0.1249, 0.0251, 0.0639, 0.1896, 0.0471, 0.0553, 0.3956, 0.0000,
         0.0952, 0.1516, 0.0479, 0.1609, 0.1435, 0.2688, 0.0024]])


In [67]:
import math

log_var2 = (math.e**log_var)

print(z)
print(mu)
print(log_var2)



(math.e**float(log_var[0][3]))**0.5

tensor([[[-0.1361,  3.0508, -2.4523,  0.1599,  0.3531,  3.7195, -2.4395,
          -1.0487, -3.2625,  1.3347,  1.1850, -2.5649, -1.6585, -0.7064,
          -5.1830,  0.4643, -2.2288, -0.5731,  0.4159,  1.6115,  1.2580,
          -0.4602,  0.0360,  0.2228, -0.1765]]], grad_fn=<UnsqueezeBackward0>)
tensor([[-0.1324,  3.0267, -2.4508,  0.1601,  0.3412,  3.7247, -2.4426, -1.0520,
         -3.2546,  1.3301,  1.1906, -2.5673, -1.6571, -0.6978, -5.1796,  0.4702,
         -2.2226, -0.5762,  0.4121,  1.6035,  1.2590, -0.4442,  0.0348,  0.2295,
         -0.1722]], grad_fn=<AddmmBackward0>)
tensor([[0.0123, 0.0256, 0.0041, 0.0050, 0.0077, 0.0072, 0.0116, 0.0048, 0.0040,
         0.0094, 0.0040, 0.0048, 0.0045, 0.0052, 0.0027, 0.0045, 0.0060, 0.0035,
         0.0044, 0.0057, 0.0047, 0.0086, 0.0043, 0.0056, 0.0036]],
       grad_fn=<PowBackward0>)


0.00495631781580328

In [78]:
print(log_var2[0][(log_var2).argmax()])
print(log_var2[0][(log_var2).argmin()])


tensor(0.0256, grad_fn=<SelectBackward0>)
tensor(0.0027, grad_fn=<SelectBackward0>)


In [82]:
normalised_variance = (log_var2 - log_var2[0][(log_var2).argmin()]) / (log_var2[0][(log_var2).argmax()] - log_var2[0][(log_var2).argmin()])


In [81]:
print(normalised_variance)

NameError: name 'normalised_variance' is not defined